In [1]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm 
from sklearn.utils import shuffle 
from collections import Counter 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import keras
from keras.utils import to_categorical
import os
from keras.layers import *
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from keras.models import Sequential, Model
from keras import optimizers

In [2]:
def parse_arff(filename):
	try:
		file = open(filename)
	except:
		print('file could not found or opened, try with file location')
	columns = []
	data = []

	for line in file:
		if line.startswith('@attribute'):
			line = line.strip()
			temp = line.split()
			columns.append(temp[1])
		elif line.startswith('@') or line.startswith('\n'):
			continue
		else:
			line = line.strip()
			temp = line.split(',')
			data.append(temp)

	return data,columns

In [3]:
data, columns = parse_arff('./data/Phishing_Legitimate_full.arff')
phishing_legitimate_full = pd.DataFrame(data, columns = columns, dtype=np.int8)
data, columns = parse_arff('./data/Training Dataset.arff')
training_dataset = pd.DataFrame(data, columns = columns, dtype=np.int8)
datasetcsv = pd.read_csv('./data/Dataset.csv')
TextFrameImage_features_new = pd.read_csv('./data/TextFrameImage Features_new.csv')
dataset_b = pd.read_csv('./data/dataset_B_05_2020.csv')
ISCXURL2016 = pd.read_csv('./data/Canadian Institute for Cybersecurity/Phishing.csv')
phishingcsv = pd.read_csv('./data/phishing.csv')

In [4]:
loss_fn = keras.losses.Huber(delta=1.0)
activation = ['softplus', 'softsign', 'selu', 'elu', 'exponential', 'tanh', 'sigmoid', 'relu']
optimizers = ['sgd', 'rmsprop', 'adam', 'adadelta', 'adagrad', 'adamax', 'nadam', 'ftrl']
binary_loss = ['binary_crossentropy', 'hinge', 'squared_hinge', loss_fn]

In [5]:
x_train, x_test, y_train, y_test = train_test_split(training_dataset[training_dataset.columns[:-1]],training_dataset[training_dataset.columns[-1]], test_size = 0.3)
y_train = np.where(y_train == -1, 0, y_train)
y_test = np.where(y_test == -1, 0, y_test)
x_train = np.array(x_train).astype(np.int8)
y_train = np.array(y_train).astype(np.int8)
x_test = np.array(x_test).astype(np.int8)
y_test = np.array(y_test).astype(np.int8)

In [9]:
model = 'safa'
model = keras.Sequential()
model.add(keras.layers.Conv2D(64,(1,1), activation='relu',input_shape=(3, 2, 5)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(16,activation='relu'))
model.add(keras.layers.Reshape((1, 16)))
model.add(keras.layers.LSTM(64, activation='relu',return_sequences=True, input_shape=(1,16)))
model.add(keras.layers.LSTM(128, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64 ,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='tanh'))

In [ ]:
x = x_train.reshape((x_train.shape[0], 3, 2, 5))
xt = x_test.reshape((x_test.shape[0], 3, 2, 5))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xt, y_test))

Epoch 1/20
242/242 [==============================] - 2s 9ms/step - loss: 0.3207 - accuracy: 0.3239 - val_loss: 0.1985 - val_accuracy: 0.4160
Epoch 2/20
242/242 [==============================] - 1s 6ms/step - loss: 0.1813 - accuracy: 0.4583 - val_loss: 0.1792 - val_accuracy: 0.4703
Epoch 3/20
242/242 [==============================] - 2s 7ms/step - loss: 0.1672 - accuracy: 0.4718 - val_loss: 0.1654 - val_accuracy: 0.4631
Epoch 4/20
242/242 [==============================] - 2s 7ms/step - loss: 0.1536 - accuracy: 0.4827 - val_loss: 0.1602 - val_accuracy: 0.4980
Epoch 5/20
242/242 [==============================] - 2s 7ms/step - loss: 0.1444 - accuracy: 0.4911 - val_loss: 0.1594 - val_accuracy: 0.5134
Epoch 6/20
242/242 [==============================] - 2s 7ms/step - loss: 0.1379 - accuracy: 0.4941 - val_loss: 0.1491 - val_accuracy: 0.4688
Epoch 7/20
242/242 [==============================] - 2s 6ms/step - loss: 0.1372 - accuracy: 0.4937 - val_loss: 0.1560 - val_accuracy: 0.5167
Epoch 

In [ ]:
model = keras.Sequential()
model.add(Dense(64, input_shape = (31,),  activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64,  activation='relu'))
model.add(Dense(1,activation='tanh'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test))

In [ ]:
x = x_train.reshape((x_train.shape[0], 1, 31))
xx = x_test.reshape((x_test.shape[0], 1, 31))

model = keras.Sequential()
model.add(keras.layers.LSTM(64, activation='relu',return_sequences=True, input_shape=(1,31)))
model.add(keras.layers.LSTM(128, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64 ,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='softplus'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xx, y_test))

In [ ]:
x = x_train.reshape((x_train.shape[0], 31, 1, 1))
xt = x_test.reshape((x_test.shape[0], 31, 1, 1))

model = keras.Sequential()
model.add(keras.layers.Conv2D(64,(1,1), activation='relu',input_shape=(31, 1, 1)))
model.add(keras.layers.MaxPool2D(1,1))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='tanh'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xt, y_test))